In [ ]:
pip install pandas scikit-learn numpy seaborn matplotlib unidecode

In [ ]:
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv('D:/Thái/NCKH/Dataset/Dataset thô/df.csv', encoding = 'utf-8-sig')

# Xem thông tin tổng quan
print(df.info())
#print(df.head())

         id                                              links  \
0  39114589  https://batdongsan.com.vn/ban-can-ho-chung-cu-...   
1  42359890  https://batdongsan.com.vn/ban-nha-biet-thu-lie...   
2  42210863  https://batdongsan.com.vn/ban-nha-biet-thu-lie...   
3  42273302  https://batdongsan.com.vn/ban-can-ho-chung-cu-...   
4  40720568  https://batdongsan.com.vn/ban-can-ho-chung-cu-...   

                                              title1  index_       price  \
0  Quỹ căn stu,1-3PN chính chủ view đẹp giá rẻ, p...       1      2,6 tỷ   
1  Độc quyền quỹ căn ngoại giao, lô góc đối diện ...       2  Thỏa thuận   
2  BÁN BIỆT THỰ, LIỀN KỀ VINHOMES ĐAN PHƯỢNG- VIN...       3     20,1 tỷ   
3  (Giá thật) Bán gấp căn hộ 3PN 107.7m2 Hà Nội P...       4     7,25 tỷ   
4  Chính chủ cần bán căn studio tòa S401, DT 25,0...       5      1,8 tỷ   

       price_per_m2      area  \
0   ~44,07 triệu/m²     59 m²   
1               NaN    200 m²   
2  ~209,38 triệu/m²     96 m²   
3   ~67,32 tri

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path

# Đọc dữ liệu
df = pd.read_csv('D:/Thái/NCKH/Dataset/Dataset thô/df.csv', encoding = 'utf-8-sig')

df['city'] = df['city'].str.strip()  # Loại bỏ khoảng trắng thừa
df['city'] = df['city'].str.title()  # Chuẩn hóa viết hoa

# Lọc các hàng có city là "Hà Nội"
df = df[df['city'] == 'Hà Nội']

########## XỬ LÍ CỘT TITLE2
# Cắt phần trước từ khoá "tại"
df['house_type'] = (
    df['title2']
      .str.split(' tại ', n=1, expand=True)[0]     # lấy phần trước “tại …”
      .str.strip()
)

# Chuẩn hoá cách viết để dễ phân tích/thống kê
def normalize(ht: str) -> str:
    h = ht.lower()
    if 'chung cư' in h:
        return 'Căn hộ chung cư'
    if 'biệt thự' in h or 'liền kề' in h:
        return 'Nhà biệt thự / liền kề'
    if 'mặt phố' in h:
        return 'Nhà mặt phố'
    if 'thương mại' in h:
        return 'Nhà phố thương mại'
    if 'riêng' in h:
        return 'Nhà riêng'
    return ht       

df['house_type'] = df['house_type'].apply(normalize)

# Tạo cột one-hot, bỏ cột gốc
df = pd.get_dummies(df,
                      columns=['house_type'],
                      drop_first=True)   # drop_first tránh đa cộng tuyến

# Danh sách các cột không cần thiết
columns_to_drop = ['links', 'title1', 'title2', 'title3', 'city', 'index_', 'id']

# Bỏ các cột
df = df.drop(columns=columns_to_drop)

########## XỬ LÍ CỘT AREA (Nhớ sửa 1,745.1)
# 1. Loại bỏ từ "m²" và khoảng trắng
df['area'] = df['area'].str.replace(' m²', '', regex=False)

# 2. Thay dấu phẩy bằng dấu chấm
df['area'] = df['area'].str.replace(',', '.', regex=False)

# 3. Chuyển đổi thành kiểu float
df['area'] = df['area'].astype(float)

########## XỬ LÍ CỘT INTERIOR
# 1: Có nội thất (không phải NaN), 0: Không có nội thất (NaN)
df['interior'] = df['interior'].notna().astype(int)

########## XỬ LÍ CỘT LEGAL
# 1: Có giấy tờ (không phải NaN), 0: Không có giấy tờ (NaN)
df['legal'] = df['legal'].notna().astype(int)

########## XỬ LSI CỘT BALCONY
# 1: Có giấy tờ (không phải NaN), 0: Không có giấy tờ (NaN)
df['balcony'] = df['balcony'].notna().astype(int)

########## XỬ LÍ CỘT DATE
# Chuyển cột date thành datetime
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df.drop(columns=['date'])

########## XỬ LÍ CỘT STREET_FRONTAGE
# Thay thế dấu phẩy bằng dấu chấm và loại bỏ ' m'
df['street_frontage'] = df['street_frontage'].str.replace(',', '.').str.replace(' m', '')

# 0: Không có mặt tiền
df['street_frontage'] = df['street_frontage'].fillna(0)

# Chuyển đổi thành kiểu float
df['street_frontage'] = pd.to_numeric(df['street_frontage'], errors='coerce')

########## XỬ LÍ CỘT DISTRICT
import pandas as pd
from unidecode import unidecode

# Xử lý cột district
# Loại bỏ khoảng trắng thừa và chuẩn hóa viết hoa
df['district'] = df['district'].str.strip()
df['district'] = df['district'].str.title()

# Bỏ dấu và xóa khoảng trắng trong cột district
df['district'] = df['district'].apply(lambda x: unidecode(x).replace(' ', ''))

# Thực hiện One-Hot Encoding
df = pd.get_dummies(df, columns=['district'], prefix='district')

########## XỬ LÍ CỘT PRICE VÀ CỘT PRICE_PER_M2
# Phát hiện nhầm lẫn và đổi chỗ dữ liệu
for idx in df.index:
    price_val = df.loc[idx, 'price']
    price_per_m2_val = df.loc[idx, 'price_per_m2']

    # Nếu cột price chứa "triệu/m²" và cột price_per_m2 chứa "tỷ", thì đổi chỗ
    if 'triệu/m²' in price_val and 'tỷ' in price_per_m2_val:
        df.loc[idx, 'price'] = price_per_m2_val
        df.loc[idx, 'price_per_m2'] = price_val

# Chuẩn hóa dữ liệu
# Hàm để bỏ chữ và chuyển thành float nhưng bỏ qua NaN
def clean_price(value):
    # Nếu giá trị là "Thoả thuận" hoặc đã là NaN, trả về NaN
    if 'Thoả thuận' in str(value) or pd.isna(value):
        return np.nan
    # Bỏ các ký tự không cần thiết
    value = str(value).replace('~', '').replace(' triệu/m²', '').replace(' tỷ', '').replace(',', '.')
    # Chuyển thành float
    try:
        return float(value)
    except ValueError:
        return np.nan  # Nếu không chuyển được, trả về NaN

# Áp dụng hàm clean_price cho cả hai cột
df['price'] = df['price'].apply(clean_price)
df['price_per_m2'] = df['price_per_m2'].apply(clean_price)

# Xử lý NaN của price và price_per_m2
# Xử lý cột price
for idx in df.index:
    price = df.loc[idx, 'price']
    price_per_m2 = df.loc[idx, 'price_per_m2']
    area = df.loc[idx, 'area']

    # Nếu price là NaN nhưng price_per_m2 không phải NaN, tính price
    if pd.isna(price) and not pd.isna(price_per_m2):
        calculated_price = price_per_m2 * area
        df.loc[idx, 'price'] = round(calculated_price, 2)  # Làm tròn đến 2 chữ số thập phân

# Xử lý cột price_per_m2
for idx in df.index:
    price = df.loc[idx, 'price']
    price_per_m2 = df.loc[idx, 'price_per_m2']
    area = df.loc[idx, 'area']

    # Nếu price_per_m2 là NaN nhưng price không phải NaN, tính price_per_m2
    if pd.isna(price_per_m2) and not pd.isna(price):
        calculated_price_per_m2 = price / area
        df.loc[idx, 'price_per_m2'] = round(calculated_price_per_m2, 2)  # Làm tròn đến 2 chữ số thập phân

#Tạo cột đánh dấu các hàng mà cả price và price_per_m2 đều là NaN
df['both_nan'] = df['price'].isna() & df['price_per_m2'].isna()

# Nếu có hàng nào mà cả price và price_per_m2 đều là NaN, tiến hành nội suy
if df['both_nan'].any():
    # Sắp xếp theo area để nội suy (có thể thay bằng cột khác như district)
    df = df.sort_values(by='area')

    # Nội suy cột price cho các hàng mà cả hai đều là NaN
    df.loc[df['both_nan'], 'price'] = df['price'].interpolate(method='linear').round(2)
    # Nội suy cột price_per_m2 cho các hàng mà cả hai đều là NaN
    df.loc[df['both_nan'], 'price_per_m2'] = df['price_per_m2'].interpolate(method='linear').round(2)

    # Nếu vẫn còn NaN (do NaN ở đầu hoặc cuối), thay bằng trung vị
    df['price'] = df['price'].fillna(df['price'].median()).round(2)
    df['price_per_m2'] = df['price_per_m2'].fillna(df['price_per_m2'].median()).round(2)

# Bỏ cột "both_nan"
df = df.drop(columns=['both_nan'])

# Sắp xếp DataFrame theo cột price
df = df.sort_values(by='price')

########## XỬ LÍ CỘT TOILETS, FLOORS, BEDROOMS
# Hàm để bỏ chữ và chuyển thành float nhưng bỏ qua NaN
def clean_value(value):
    # Bỏ các ký tự không cần thiết
    value = str(value).replace(' phòng', '').replace(' tầng', '')
    # Chuyển thành float
    try:
        return float(value)
    except ValueError:
        return np.nan  # Nếu không chuyển được, trả về NaN

# Áp dụng hàm clean_price cho cả hai cột
df['toilets'] = df['toilets'].apply(clean_value)
df['floors'] = df['floors'].apply(clean_value)
df['bedrooms'] = df['bedrooms'].apply(clean_value)

# Xử lý NaN của toilets và floors và bedrooms
# Hàm nội suy và làm tròn theo nguyên tắc (>= 0.5 thì tăng, < 0.5 thì giữ nguyên)
def interpolate_and_round(column):
    if column.isna().any():
        # Nội suy tuyến tính
        column = column.interpolate(method='linear')
        # Làm tròn theo nguyên tắc mặc định của round()
        column = column.round()
        # Điền các giá trị NaN còn lại bằng median (cũng làm tròn)
        column = column.fillna(round(column.median()))
    return column

# Áp dụng nội suy và làm tròn cho từng cột
df['toilets'] = interpolate_and_round(df['toilets'])
df['floors'] = interpolate_and_round(df['floors'])
df['bedrooms'] = interpolate_and_round(df['bedrooms'])

df = df.sort_values(by='year')

# Lưu DataFrame vào file CSV
df.to_csv('data_processing.csv', index=False)
print("DataFrame đã được lưu vào file 'data_processing.csv'")

DataFrame đã được lưu vào file 'data_processing.csv'


In [10]:
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv('D:/Thái/NCKH/Code làm sạch dữ liệu/data_processing.csv', encoding = 'utf-8-sig')

# Xem thông tin tổng quan
#print(df.info())
print(df.head())

   price  price_per_m2   area  floors  toilets  bedrooms  balcony  \
0  53.67        370.11  145.0     6.0      2.0       3.0        1   
1  44.19        312.50  141.4     5.0      4.0       5.0        1   
2  28.60        381.33   75.0     6.0      8.0       8.0        1   
3  23.35        389.16   60.0     5.0      2.0       2.0        1   
4  20.90        278.67   75.0     6.0      2.0       3.0        0   

   street_frontage  legal  interior  ...  district_QuocOai  district_SocSon  \
0              0.0      1         0  ...             False            False   
1             20.0      1         1  ...             False            False   
2              5.0      1         1  ...             False            False   
3              0.0      1         1  ...             False            False   
4              0.0      1         1  ...             False            False   

   district_SonTay  district_TayHo  district_ThachThat  district_ThanhOai  \
0            False           Fals